In [1]:
import os
import sys
sys.path.append("./ddn/")
sys.path.append("./")
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import scipy.special
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from scipy.linalg import block_diag
from torch.utils.data import Dataset, DataLoader
#from bernstein import bernstesin_coeff_order10_new
from utils import *
from ddn.pytorch.node import AbstractDeclarativeNode

In [2]:
!TMPDIR=/home/vikrant pip install --cache-dir=/home/vikrant/ --build /home/vikrant/ pandas

#### CUDA Initializations

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [4]:
def bernstein_coeff_order10_new(n, tmin, tmax, t_actual):
    l = tmax - tmin
    t = (t_actual - tmin) / l

    P0 = scipy.special.binom(n, 0) * ((1 - t) ** (n - 0)) * t ** 0
    P1 = scipy.special.binom(n, 1) * ((1 - t) ** (n - 1)) * t ** 1
    P2 = scipy.special.binom(n, 2) * ((1 - t) ** (n - 2)) * t ** 2
    P3 = scipy.special.binom(n, 3) * ((1 - t) ** (n - 3)) * t ** 3
    P4 = scipy.special.binom(n, 4) * ((1 - t) ** (n - 4)) * t ** 4
    P5 = scipy.special.binom(n, 5) * ((1 - t) ** (n - 5)) * t ** 5
    P6 = scipy.special.binom(n, 6) * ((1 - t) ** (n - 6)) * t ** 6
    P7 = scipy.special.binom(n, 7) * ((1 - t) ** (n - 7)) * t ** 7
    P8 = scipy.special.binom(n, 8) * ((1 - t) ** (n - 8)) * t ** 8
    P9 = scipy.special.binom(n, 9) * ((1 - t) ** (n - 9)) * t ** 9
    P10 = scipy.special.binom(n, 10) * ((1 - t) ** (n - 10)) * t ** 10

    P0dot = -10.0 * (-t + 1) ** 9
    P1dot = -90.0 * t * (-t + 1) ** 8 + 10.0 * (-t + 1) ** 9
    P2dot = -360.0 * t ** 2 * (-t + 1) ** 7 + 90.0 * t * (-t + 1) ** 8
    P3dot = -840.0 * t ** 3 * (-t + 1) ** 6 + 360.0 * t ** 2 * (-t + 1) ** 7
    P4dot = -1260.0 * t ** 4 * (-t + 1) ** 5 + 840.0 * t ** 3 * (-t + 1) ** 6
    P5dot = -1260.0 * t ** 5 * (-t + 1) ** 4 + 1260.0 * t ** 4 * (-t + 1) ** 5
    P6dot = -840.0 * t ** 6 * (-t + 1) ** 3 + 1260.0 * t ** 5 * (-t + 1) ** 4
    P7dot = -360.0 * t ** 7 * (-t + 1) ** 2 + 840.0 * t ** 6 * (-t + 1) ** 3
    P8dot = 45.0 * t ** 8 * (2 * t - 2) + 360.0 * t ** 7 * (-t + 1) ** 2
    P9dot = -10.0 * t ** 9 + 9 * t ** 8 * (-10.0 * t + 10.0)
    P10dot = 10.0 * t ** 9

    P0ddot = 90.0 * (-t + 1) ** 8
    P1ddot = 720.0 * t * (-t + 1) ** 7 - 180.0 * (-t + 1) ** 8
    P2ddot = 2520.0 * t ** 2 * (-t + 1) ** 6 - 1440.0 * t * (-t + 1) ** 7 + 90.0 * (-t + 1) ** 8
    P3ddot = 5040.0 * t ** 3 * (-t + 1) ** 5 - 5040.0 * t ** 2 * (-t + 1) ** 6 + 720.0 * t * (-t + 1) ** 7
    P4ddot = 6300.0 * t ** 4 * (-t + 1) ** 4 - 10080.0 * t ** 3 * (-t + 1) ** 5 + 2520.0 * t ** 2 * (-t + 1) ** 6
    P5ddot = 5040.0 * t ** 5 * (-t + 1) ** 3 - 12600.0 * t ** 4 * (-t + 1) ** 4 + 5040.0 * t ** 3 * (-t + 1) ** 5
    P6ddot = 2520.0 * t ** 6 * (-t + 1) ** 2 - 10080.0 * t ** 5 * (-t + 1) ** 3 + 6300.0 * t ** 4 * (-t + 1) ** 4
    P7ddot = -360.0 * t ** 7 * (2 * t - 2) - 5040.0 * t ** 6 * (-t + 1) ** 2 + 5040.0 * t ** 5 * (-t + 1) ** 3
    P8ddot = 90.0 * t ** 8 + 720.0 * t ** 7 * (2 * t - 2) + 2520.0 * t ** 6 * (-t + 1) ** 2
    P9ddot = -180.0 * t ** 8 + 72 * t ** 7 * (-10.0 * t + 10.0)
    P10ddot = 90.0 * t ** 8
    90.0 * t ** 8

    P = np.hstack((P0, P1, P2, P3, P4, P5, P6, P7, P8, P9, P10))
    Pdot = np.hstack((P0dot, P1dot, P2dot, P3dot, P4dot, P5dot, P6dot, P7dot, P8dot, P9dot, P10dot)) / l
    Pddot = np.hstack((P0ddot, P1ddot, P2ddot, P3ddot, P4ddot, P5ddot, P6ddot, P7ddot, P8ddot, P9ddot, P10ddot)) / (l ** 2)
    return P, Pdot, Pddot

In [5]:
def plot_traj(cnt, traj_inp, traj_out, traj_pred, obs):
    traj_inp = traj_inp.numpy()[:2*num]
    traj_out = traj_out.numpy()
    traj_pred = traj_pred.detach().numpy()
    
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.scatter(traj_inp[::2], traj_inp[1::2], label='Inp traj')
    ax.scatter(traj_out[:num], traj_out[num:], label='GT')
    ax.scatter(traj_pred[:num], traj_pred[num:], label='Pred')
#     ax.scatter(obs["x"], obs["y"], s=20, label="Obstacles")
    
    x_obs_temp = obs["x"]
    y_obs_temp = obs["y"]
    
    th = np.linspace(0, 2 * np.pi, 100)
    for i in range(0, num_obs):
        x_circ = x_obs_temp[i] + a_obs * np.cos(th)
        y_circ = y_obs_temp[i] + b_obs * np.sin(th)
        ax.plot(x_circ, y_circ, '-k')
    
    ax.legend()
    ax.set_xlim([-7, 7])
    ax.set_ylim([-7, 7])
    plt.savefig('./results/{}.png'.format(cnt))
    plt.close()

In [6]:
# rho_obs = 0.3
rho_obs = 1.2
rho_eq = 10.0
weight_smoothness = 10

In [7]:
num = 10
t_fin = 2
a_obs = 1.0
b_obs = 1.0

tot_time = np.linspace(0.0, t_fin, num)
tot_time_copy = tot_time.reshape(num, 1)
P, Pdot, Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)
nvar = np.shape(P)[1]

In [8]:
# x_obs_temp = np.hstack((-10.0, -10.79, -10, -10))
# y_obs_temp = np.hstack((10.0, 15.0, 15, 15))
x_obs_temp = np.hstack((-2.0, -5.79, 3.0, 4.0))
y_obs_temp = np.hstack((-2.0, 2.0, -0.80, 2.0))
num_obs = np.shape(x_obs_temp)[0]

x_obs = np.ones((num_obs, num)) * x_obs_temp[:, np.newaxis]
y_obs = np.ones((num_obs, num)) * y_obs_temp[:, np.newaxis]
x_obs
y_obs

array([[-2. , -2. , -2. , -2. , -2. , -2. , -2. , -2. , -2. , -2. ],
       [ 2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ],
       [-0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.8],
       [ 2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ]])

In [9]:
obs_inp_list = []
for i in range(num_obs):
    obs_inp_list.extend([x_obs_temp[i], y_obs_temp[i], a_obs])

obs_inp = np.array(obs_inp_list)
obs_inp

array([-2.  , -2.  ,  1.  , -5.79,  2.  ,  1.  ,  3.  , -0.8 ,  1.  ,
        4.  ,  2.  ,  1.  ])

In [10]:
A_obs = np.tile(P, (num_obs, 1))
A_eq = np.vstack((P[0], Pdot[0], Pddot[0], P[-1], Pdot[-1], Pddot[-1]))
Q_smoothness = np.dot(Pddot.T, Pddot)

In [11]:
# A_obs

In [12]:
#plot({"x":x, "y":y}, {"x":x_obs_temp, "y":y_obs_temp})

In [13]:
class OPTNode(AbstractDeclarativeNode):
    def __init__(self, rho_eq=1.0, rho_goal=1.0, rho_lane=1.0, rho_nonhol=1.0, rho_w_psi=1.0, rho_psi=1.0, maxiter=500, weight_smoothness=1.0, weight_smoothness_psi=1.0, t_fin=8.0, num=100, rho_mid=0.01):
        super().__init__()
        self.rho_eq = rho_eq
        self.rho_goal = rho_goal
        self.rho_lane = rho_lane
        self.rho_nonhol = rho_nonhol
        self.rho_w_psi = rho_w_psi
        self.rho_psi = rho_psi
        self.maxiter = maxiter
        self.weight_smoothness = weight_smoothness
        self.weight_smoothness_psi = weight_smoothness_psi

        self.t_fin = t_fin
        self.num = num
        self.t = self.t_fin / self.num

        tot_time = torch.linspace(0.0, self.t_fin, self.num).to(device)
        tot_time_copy = tot_time.reshape(self.num, 1)
        self.P, self.Pdot, self.Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)

        self.nvar = np.shape(self.P)[1]
        self.A_eq_psi = np.vstack((self.P[0], self.Pdot[0], self.P[-1], self.Pdot[-1]))

        self.cost_smoothness = self.weight_smoothness * np.dot(self.Pddot.T, self.Pddot)
        self.cost_smoothness_v = self.weight_smoothness * np.dot(self.Pddot.T, self.Pddot)
        self.cost_smoothness_psi = self.weight_smoothness_psi * np.dot(self.Pddot.T, self.Pddot)
        self.lincost_smoothness_psi = np.zeros(self.nvar)

        self.P = torch.tensor(self.P, dtype=torch.double).to(device)
        self.Pdot = torch.tensor(self.Pdot, dtype=torch.double).to(device)
        self.Pddot = torch.tensor(self.Pddot, dtype=torch.double).to(device)
        self.A_eq_psi = torch.tensor(self.A_eq_psi, dtype=torch.double).to(device)
        self.cost_smoothness = torch.tensor(self.cost_smoothness, dtype=torch.double).to(device)
        self.cost_smoothness_v = torch.tensor(self.cost_smoothness_v, dtype=torch.double).to(device)
        self.cost_smoothness_psi = torch.tensor(self.cost_smoothness_psi, dtype=torch.double).to(device)
        self.lincost_smoothness_psi = torch.tensor(self.lincost_smoothness_psi, dtype=torch.double).to(device)

        self.rho_mid = rho_mid
        self.mid_idx = torch.tensor([int(self.num/4), int(self.num/2), int(3*self.num/4)]).to(device)
        
        self.A_w = self.P
        self.A_w_psi = self.P
        self.A_psi = self.P

    def compute_w_psi(self, x_init, y_init, x_fin, y_fin, x_mid, y_mid, psi, v, lamda_wc, lamda_ws):
        b_wc_psi = torch.cos(psi).double()
        b_ws_psi = torch.sin(psi).double()

        temp_x = torch.cumsum(self.P * (v * self.t)[:, np.newaxis], axis=0)
        temp_y = torch.cumsum(self.P * (v * self.t)[:, np.newaxis], axis=0)

        A_x = temp_x[0:self.num - 1]
        A_y = temp_y[0:self.num - 1]

        A_x_goal = A_x[-1].reshape(1, self.nvar)
        b_x_goal = torch.tensor([x_fin - x_init], dtype=torch.double)

        A_y_goal = A_y[-1].reshape(1, self.nvar)
        b_y_goal = torch.tensor([y_fin - y_init], dtype=torch.double)

        A_x_mid = A_x[self.mid_idx]
        A_y_mid = A_y[self.mid_idx]

        b_x_mid = x_mid - x_init
        b_y_mid = y_mid - y_init

        obj_x_goal = self.rho_goal * torch.matmul(A_x_goal.T, A_x_goal)
        linterm_augment_x_goal = -self.rho_goal * torch.matmul(A_x_goal.T, b_x_goal)

        obj_y_goal = self.rho_goal * torch.matmul(A_y_goal.T, A_y_goal)
        linterm_augment_y_goal = -self.rho_goal * torch.matmul(A_y_goal.T, b_y_goal)

        obj_x_mid = self.rho_mid * torch.matmul(A_x_mid.T, A_x_mid)
        linterm_augment_x_mid = -self.rho_mid * torch.matmul(A_x_mid.T, b_x_mid)

        obj_y_mid = self.rho_mid * torch.matmul(A_y_mid.T, A_y_mid)
        linterm_augment_y_mid = -self.rho_mid * torch.matmul(A_y_mid.T, b_y_mid)

        obj_wc_psi = self.rho_w_psi * torch.matmul(self.A_w_psi.T, self.A_w_psi)
        linterm_augment_wc_psi = -self.rho_w_psi * torch.matmul(self.A_w_psi.T, b_wc_psi)

        obj_ws_psi = self.rho_w_psi * torch.matmul(self.A_w_psi.T, self.A_w_psi)
        linterm_augment_ws_psi = -self.rho_w_psi * torch.matmul(self.A_w_psi.T, b_ws_psi)

        cost_wc = obj_wc_psi + obj_x_goal + obj_x_mid
        lincost_wc = -lamda_wc + linterm_augment_x_goal + linterm_augment_wc_psi + linterm_augment_x_mid

        cost_ws = obj_y_goal + obj_ws_psi + obj_y_mid
        lincost_ws = -lamda_ws + linterm_augment_y_goal + linterm_augment_ws_psi + linterm_augment_y_mid

        c_wc_psi = torch.linalg.solve(-cost_wc, lincost_wc)
        c_ws_psi = torch.linalg.solve(-cost_ws, lincost_ws)

        wc = torch.matmul(self.P, c_wc_psi)
        ws = torch.matmul(self.P, c_ws_psi)

        return wc, ws, c_wc_psi, c_ws_psi

    def compute_psi(self, wc, ws, b_eq_psi, lamda_psi):
        b_psi = torch.atan2(ws, wc).double()
        obj_psi = self.rho_psi * torch.matmul(self.A_psi.T, self.A_psi)
        linterm_augment_psi = -self.rho_psi * torch.matmul(self.A_psi.T, b_psi)

        cost_psi = self.cost_smoothness_psi + obj_psi + self.rho_eq * torch.matmul(self.A_eq_psi.T, self.A_eq_psi)
        lincost_psi = -lamda_psi + linterm_augment_psi - self.rho_eq * torch.matmul(self.A_eq_psi.T, b_eq_psi)
        sol = torch.linalg.solve(-cost_psi, lincost_psi)

        c_psi = sol[0:self.nvar]
        psi = torch.matmul(self.P, c_psi)

        res_psi = torch.matmul(self.A_psi, c_psi) - b_psi
        res_eq_psi = torch.matmul(self.A_eq_psi, c_psi) - b_eq_psi
        lamda_psi = lamda_psi - self.rho_psi * torch.matmul(self.A_psi.T, res_psi) - self.rho_eq * torch.matmul(self.A_eq_psi.T, res_eq_psi)
        return psi, c_psi, torch.linalg.norm(res_psi), torch.linalg.norm(res_eq_psi), lamda_psi
    
    def compute_v(self, v_init, x_init, x_fin, x_mid, y_init, y_fin, y_mid, psi, lamda_v):
        temp_x = torch.cumsum(self.P * (torch.cos(psi) * self.t)[:, np.newaxis], axis=0)
        temp_y = torch.cumsum(self.P * (torch.sin(psi) * self.t)[:, np.newaxis], axis=0)

        A_x = temp_x[0:self.num - 1]
        A_y = temp_y[0:self.num - 1]

        A_x_goal = A_x[-1].resize(1, self.nvar)
        b_x_goal = (x_fin - x_init).unsqueeze(0)

        A_y_goal = A_y[-1].resize(1, self.nvar)
        b_y_goal = (y_fin - y_init).unsqueeze(0)

        A_x_mid = A_x[self.mid_idx]
        A_y_mid = A_y[self.mid_idx]

        b_x_mid = x_mid - x_init
        b_y_mid = y_mid - y_init

        A_vel_init = self.P[0].reshape(1, self.nvar)
        b_vel_init = torch.tensor([v_init], dtype=torch.double)

        obj_x_goal = self.rho_goal * torch.matmul(A_x_goal.T, A_x_goal)
        linterm_augment_x_goal = -self.rho_goal * torch.matmul(A_x_goal.T, b_x_goal)

        obj_y_goal = self.rho_goal * torch.matmul(A_y_goal.T, A_y_goal)
        linterm_augment_y_goal = -self.rho_goal * torch.matmul(A_y_goal.T, b_y_goal)

        obj_x_mid = self.rho_mid * torch.matmul(A_x_mid.T, A_x_mid)
        linterm_augment_x_mid = -self.rho_mid * torch.matmul(A_x_mid.T, b_x_mid)

        obj_y_mid = self.rho_mid * torch.matmul(A_y_mid.T, A_y_mid)
        linterm_augment_y_mid = -self.rho_mid * torch.matmul(A_y_mid.T, b_y_mid)

        obj_v_init = self.rho_eq * torch.matmul(A_vel_init.T, A_vel_init)
        linterm_augment_v_init = -self.rho_eq * torch.matmul(A_vel_init.T, b_vel_init)

        cost = obj_x_goal + obj_y_goal + self.cost_smoothness_v + obj_x_mid + obj_y_mid + obj_v_init
        lincost = -lamda_v + linterm_augment_x_goal + linterm_augment_y_goal + linterm_augment_x_mid + linterm_augment_y_mid + linterm_augment_v_init

        sol = torch.linalg.solve(-cost, lincost)

        v = torch.matmul(self.P, sol)

        res_v_init = torch.matmul(A_vel_init, sol) - b_vel_init
        lamda_v = lamda_v - self.rho_eq * torch.matmul(A_vel_init.T, res_v_init)

        return sol, lamda_v, v
    
    def optimize(self, fixed_params, variable_params):
        x_init, y_init, v_init, psi_init, psidot_init = fixed_params
        x_fin, y_fin, v_fin, psi_fin, psidot_fin = variable_params[:5]
        x_mid = variable_params[5::2]
        y_mid = variable_params[6::2]

        v = v_init * torch.ones(self.num, dtype=torch.double).to(device)
        psi = psi_init * torch.ones(self.num, dtype=torch.double).to(device)

        res_psi = torch.ones(self.maxiter, dtype=torch.double).to(device)
        res_w_psi = torch.ones(self.maxiter, dtype=torch.double).to(device)
        res_w = torch.ones(self.maxiter, dtype=torch.double).to(device)
        res_eq_psi = torch.ones(self.maxiter, dtype=torch.double).to(device)
        res_eq = torch.ones(self.maxiter, dtype=torch.double).to(device)
        b_eq_psi = torch.hstack((psi_init, psidot_init, psi_fin, psidot_fin))

        lamda_wc = torch.zeros(self.nvar, dtype=torch.double).to(device)
        lamda_ws = torch.zeros(self.nvar, dtype=torch.double).to(device)
        lamda_psi = torch.zeros(self.nvar, dtype=torch.double).to(device)
        lamda_v = torch.zeros(self.nvar, dtype=torch.double).to(device)
        
        for i in range(0, self.maxiter):
            wc, ws, c_wc_psi, c_ws_psi = self.compute_w_psi(x_init, y_init, x_fin, y_fin, x_mid, y_mid, psi, v, lamda_wc, lamda_ws)
            psi, c_psi, res_psi[i], res_eq_psi[i], lamda_psi = self.compute_psi(wc, ws, b_eq_psi, lamda_psi)
            c_v, lamda_v, v = self.compute_v(v_init, x_init, x_fin, x_mid, y_init, y_fin, y_mid, psi, lamda_v)

            res_wc = wc - torch.cos(psi)
            res_ws = ws - torch.sin(psi)
            
            lamda_wc = lamda_wc - self.rho_w_psi * torch.matmul(self.A_w.T, res_wc)
            lamda_ws = lamda_ws - self.rho_w_psi * torch.matmul(self.A_w.T, res_ws)

            res_w[i] = torch.linalg.norm(torch.hstack((res_wc, res_ws)))
            
        primal_sol = torch.hstack((c_psi, c_v))
        return primal_sol
    
    def solve(self, fixed_params, variable_params):
        batch_size, _ = fixed_params.size()
        y = torch.zeros(batch_size, 2 * self.nvar, dtype=torch.double).to(device)
        mid_size = self.mid_idx.size()[0]
        for i in range(batch_size):
            fixed_params_cur = fixed_params[i]
            variable_params_cur = variable_params[i]
            primal_sol = self.optimize(fixed_params_cur, variable_params_cur)
            y[i, :] = primal_sol
        return y, None
    
    def objective(self, fixed_params, variable_params, y):
        x_init, y_init, v_init, psi_init, psidot_init = torch.chunk(fixed_params, 5, dim=1)
        x_fin, y_fin, v_fin, psi_fin, psidot_fin = torch.chunk(variable_params[:, :5], 5, dim=1)
        x_mid = variable_params[:, 5::2]
        y_mid = variable_params[:, 6::2]
        
        c_psi = y[:, :self.nvar]
        c_v = y[:, self.nvar:]
        
        v_new = torch.matmul(self.P, c_v.T)
        psi_new = torch.matmul(self.P, c_psi.T)
        psidot_new = torch.matmul(self.Pdot, c_psi.T)
        
        x_temp = x_init + torch.cumsum(v_new * torch.cos(psi_new) * self.t, dim=0).T
        y_temp = y_init + torch.cumsum(v_new * torch.sin(psi_new) * self.t, dim=0).T
        
        x = torch.hstack((x_init, x_temp[:, :-1]))
        y = torch.hstack((y_init, y_temp[:, :-1]))
        
        cost_final_pos = 0.5 * self.rho_goal * ((x[:, -1:] - x_fin) ** 2 + (y[:, -1:] - y_fin) ** 2)
        cost_psi_term = 0.5 * self.rho_eq * ((psi_new[:1, :].T - psi_init) ** 2 + (psidot_new[:1, :].T - psidot_init) ** 2 + (psi_new[-1:, :].T - psi_fin) ** 2 + (psidot_new[-1:, :].T - psidot_fin) ** 2)
        cost_v_term = 0.5 * self.rho_eq * (v_new[:1, :].T - v_init) ** 2
        cost_mid_term = 0.5 * self.rho_mid * (torch.sum((x[:, self.mid_idx] - x_mid) ** 2, dim=1, keepdim=True) + torch.sum((y[:, self.mid_idx] - y_mid) ** 2, dim=1, keepdim=True))
        
        cost = cost_final_pos + cost_psi_term + cost_v_term + cost_mid_term
        return cost

#### PyTorch Declarative Function

In [14]:
class OptFunction(torch.autograd.Function):
    """Generic declarative autograd function.
    Defines the forward and backward functions. Saves all inputs and outputs,
    which may be memory-inefficient for the specific problem.
    
    Assumptions:
    * All inputs are PyTorch tensors
    * All inputs have a single batch dimension (b, ...)
    """
    @staticmethod
    def forward(ctx, problem, *inputs):
        output, solve_ctx = torch.no_grad()(problem.solve)(*inputs)
        ctx.save_for_backward(output, *inputs)
        ctx.problem = problem
        ctx.solve_ctx = solve_ctx
        return output.clone()

    @staticmethod
    def backward(ctx, grad_output):
        output, *inputs = ctx.saved_tensors
        problem = ctx.problem
        solve_ctx = ctx.solve_ctx
        output.requires_grad = True
        inputs = tuple(inputs)
        grad_inputs = problem.gradient(*inputs, y=output, v=grad_output,
            ctx=solve_ctx)
        return (None, *grad_inputs)

#### PyTorch Declarative Layer

In [15]:
class OptLayer(torch.nn.Module):
    """Generic declarative layer.
    
    Assumptions:
    * All inputs are PyTorch tensors
    * All inputs have a single batch dimension (b, ...)
    Usage:
        problem = <derived class of *DeclarativeNode>
        declarative_layer = DeclarativeLayer(problem)
        y = declarative_layer(x1, x2, ...)
    """
    def __init__(self, problem):
        super(OptLayer, self).__init__()
        self.problem = problem
        
    def forward(self, *inputs):
        return OptFunction.apply(self.problem, *inputs)

#### TrajNet

In [16]:
class TrajNet(nn.Module):
    def __init__(self, opt_layer, P, input_size=16, hidden_size=64, output_size=12, nvar=11, t_obs=8):
        super(TrajNet, self).__init__()
        self.P = torch.tensor(P, dtype=torch.double).to(device)
        self.nvar = nvar
        self.t_obs = t_obs
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        self.opt_layer = opt_layer
        self.activation = nn.ReLU()
        self.mask = torch.tensor([[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]], dtype=torch.double)
    
    def forward(self, x, b):
        batch_size, _ = x.size()
        out = self.activation(self.linear1(x))
        b_pred = self.linear2(out)
        b_gen = self.mask * b + (1 - self.mask) * b_pred
        
        # Run optimization
        lamda_x = torch.zeros(batch_size, self.nvar, dtype=torch.double).to(device)
        lamda_y = torch.zeros(batch_size, self.nvar, dtype=torch.double).to(device)
        sol = self.opt_layer(b_gen, lamda_x, lamda_y)

        # Compute final trajectory
        x_pred = torch.matmul(self.P, sol[:, :self.nvar].transpose(0, 1))
        y_pred = torch.matmul(self.P, sol[:, self.nvar:2*self.nvar].transpose(0, 1))
        
        x_pred = x_pred.transpose(0, 1)
        y_pred = y_pred.transpose(0, 1)
        out = torch.cat([x_pred, y_pred], dim=1)
        #return out, sol, b_gen, lamda_x, lamda_y
        return out

In [26]:
opt_node = OPTNode(
    P, 
    Pddot, 
    A_eq,
    A_obs, 
    Q_smoothness, 
    x_obs,
    y_obs,
    num=num, 
#     num_obs=num_obs, 
#     nvar=nvar, 
#     a_obs=a_obs, 
#     b_obs=b_obs, 
#     rho_obs=rho_obs, 
#     rho_eq=rho_eq,
    weight_smoothness=weight_smoothness, 
#     maxiter=300, 
#     eps=1e-7, 
#     num_tot=num*num_obs
)
opt_layer = OptLayer(opt_node)

In [27]:
model = TrajNet(opt_layer, P, 36)
model = model.double()
model = model.to(device)

#### Model Testing

In [28]:
x_inp = torch.randn(10, 36, dtype=torch.double)
y_out = torch.randn(10, 20, dtype=torch.double)
b_inp = torch.randn(10, 12, dtype=torch.double)

In [29]:
print(x_inp.shape)

torch.Size([10, 36])


In [30]:
out = model(x_inp, b_inp)
out.size()

TypeError: solve() takes 3 positional arguments but 4 were given

In [83]:
batch_size, _ = x_inp.size()
lamda_x = torch.zeros(batch_size, nvar, dtype=torch.double).to(device)
lamda_y = torch.zeros(batch_size, nvar, dtype=torch.double).to(device)
opt_node.gradient(b_inp, lamda_x, lamda_y)

(None, None, None)

#### Trajectory DataLoader

In [44]:
import pandas as pd
class PedDataLoader(Dataset):    
    def __init__(self, file_path, threshold=20):        
        """        
            Takes the filepath of input zara dataset, and trajectory length threshold.        
        """        
        self.df = pd.read_csv(file_path, sep='\t', header=None, names=["Frame", "Agent", "x-pos", "y-pos"]) 
        print(self.df)
        self.data = []    
        self.dt = 0.2
        cnt = 0
        for i in range(int(self.df["Agent"].max())):              
            self.data.append([])
            cnt = cnt + 1
            for row in self.df.itertuples():
                if int(row[2] - 1) == i and isinstance(self.data[int(row[2]) - 1],list) == False:
                    self.data[int(row[2] - 1)].append([])
                if int(row[2] - 1) == i and 1 and len(self.data[int(row[2] - 1)]) <= threshold:
                    self.data[int(row[2] - 1)].append([row[3], row[4]])
        self.data = list(filter(lambda x: len(x)==threshold, self.data))    
        print(np.array(self.data).shape)
#         print(np.array(self.data))
    def __len__(self):        
        return len(self.data)                
    
    def __getitem__(self, ind):       
        traj = np.array(self.data[ind])
        print(traj.shape)
        x_traj = traj[:, 0]
        y_traj = traj[:, 1]        
#         print(x_traj, y_traj)       
        x_inp = x_traj[:10]        
        x_fut = x_traj[10:]        
        y_inp = y_traj[:10]        
        y_fut = y_traj[10:]               
        vx_beg = x_inp[-1] - x_inp[-2]        
        vy_beg = y_inp[-1] - y_inp[-2]        
        vel_acc = np.array([vx_beg, vy_beg, 0, 0])        
        traj_inp = np.dstack((x_inp, y_inp)).flatten()        
        traj_inp = np.hstack((traj_inp, vel_acc, obs_inp))        
        traj_out = np.hstack((x_fut, y_fut)).flatten()        
        b_inp = np.array([x_fut[0], vx_beg, 0, x_fut[-1], 0, 0, y_fut[0], vy_beg, 0, y_fut[-1], 0, 0])    
        return torch.tensor(traj_inp), torch.tensor(traj_out), torch.tensor(b_inp)

class TrajectoryDataset(Dataset):
    def __init__(self, root_dir, t_obs=10, dt=0.4):
        self.root_dir = root_dir
        self.t_obs = t_obs
        self.dt = dt
        self.data = []
        data = np.load(self.root_dir + "/test.npy")
        print(data.shape)
        self.data = data    
    
    def __len__(self):
        return len(self.data)
    
    def get_vel(self, pos):
        return (pos[-1] - pos[-2]) / self.dt
    
    def get_acc(self, vel):
        return (vel[-1] - vel[-2]) / self.dt
    
    def __getitem__(self, idx):
#         file_name = "tests/test.npy"
#         file_path = os.path.join(self.root_dir, file_name)
        
        data = np.load(self.root_dir + "/test.npy")
        self.data = data
#         print(data[idx].shape)
#         print(data[idx][0])
        x_traj = data[idx][:, 0]
        y_traj = data[idx][:, 1]
        
        x_inp = x_traj[:self.t_obs]
        y_inp = y_traj[:self.t_obs]
        x_fut = x_traj[self.t_obs:]
        y_fut = y_traj[self.t_obs:]
        
        vx_beg = x_inp[-1] - x_inp[-2]        
        vy_beg = y_inp[-1] - y_inp[-2]        
        vel_acc = np.array([vx_beg, vy_beg, 0, 0])        
        traj_inp = np.dstack((x_inp, y_inp)).flatten()        
        traj_inp = np.hstack((traj_inp, vel_acc, obs_inp))        
        traj_out = np.hstack((x_fut, y_fut)).flatten()
#         print(x_fut, y_fut)
        b_inp = np.array([x_fut[0], vx_beg, 0, x_fut[-1], 0, 0, y_fut[0], vy_beg, 0, y_fut[-1], 0, 0])    
        return torch.tensor(traj_inp), torch.tensor(traj_out), torch.tensor(b_inp)

In [45]:
# train_dataset = PedDataLoader("./datasets/zara1/train/students003_train.txt")
# train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=0)

In [46]:
train_dataset = TrajectoryDataset("./tests")
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=0)

(100, 20, 2)


In [47]:
# test_dataset = TrajectoryDataset("./test2/data/", t_obs=25, dt=0.08)
# test_loader = DataLoader(test_dataset, batch_size=20, shuffle=True, num_workers=0)

In [48]:
for i, data in enumerate(train_loader):
    traj_inp, traj_out, b_inp = data
    print(traj_inp.size(), traj_out.size(), b_inp.size())
    break

torch.Size([20, 36]) torch.Size([20, 20]) torch.Size([20, 12])


In [49]:
# out = model(traj_inp, b_inp)
# out.size()

#### Training

In [50]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [51]:
traj_inp.size(), traj_out.size()

(torch.Size([20, 36]), torch.Size([20, 20]))

In [52]:
epoch_train_loss = []
num_epochs = 25

for epoch in range(num_epochs):
    train_loss = []
    for batch_num, data in enumerate(train_loader):
        traj_inp, traj_out, b_inp = data
        traj_inp = traj_inp.to(device)
        traj_out = traj_out.to(device)
        b_inp = b_inp.to(device)

        out = model(traj_inp, b_inp)
        loss = criterion(out, traj_out)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
        
        cnt = 0
        obs = np.load("train/obstacle.npy")
        for i in range(traj_inp.size()[0]):
            plot_traj(cnt, traj_inp[i], traj_out[i], out[i], {"x": x_obs_temp, "y": y_obs_temp})
            cnt += 1
        
        if batch_num % 10 == 0:
            print("Epoch: {}, Batch: {}, Loss: {}".format(epoch, batch_num, loss.item()))
    
    mean_loss = np.mean(train_loss)
    epoch_train_loss.append(mean_loss)
    print("Epoch: {}, Mean Loss: {}".format(epoch, mean_loss))
    print("-"*100)

Epoch: 0, Batch: 0, Loss: 3.0481331363347306
Epoch: 0, Mean Loss: 1.7811489985744289
----------------------------------------------------------------------------------------------------
Epoch: 1, Batch: 0, Loss: 1.217869182545357
Epoch: 1, Mean Loss: 1.0416578574414712
----------------------------------------------------------------------------------------------------
Epoch: 2, Batch: 0, Loss: 0.7797071260774974
Epoch: 2, Mean Loss: 0.9119967196240559
----------------------------------------------------------------------------------------------------
Epoch: 3, Batch: 0, Loss: 0.9364042018108267
Epoch: 3, Mean Loss: 0.8978284588835728
----------------------------------------------------------------------------------------------------
Epoch: 4, Batch: 0, Loss: 0.9738340806820664
Epoch: 4, Mean Loss: 0.8886460951635582
----------------------------------------------------------------------------------------------------
Epoch: 5, Batch: 0, Loss: 0.9683644807741638
Epoch: 5, Mean Loss: 0.885

#### Testing

In [ ]:
plot_traj(1, traj_inp, traj_out, traj_pred)

In [ ]:
with torch.no_grad():
    cnt = 0
    test_loss = []
    for batch_num, data in enumerate(test_loader):
        traj_inp, traj_out, b_inp = data
        traj_inp = traj_inp.to(device)
        traj_out = traj_out.to(device)
        b_inp = b_inp.to(device)

        out = model(traj_inp, b_inp)
        loss = criterion(out, traj_out)
        
        test_loss.append(loss.item())
        obs = np.load("train/obstacle.npy")
        print("Batch: {}, Loss: {}".format(batch_num, loss.item()))
        for i in range(traj_inp.size()[0]):
            plot_traj(cnt, traj_inp[i], traj_out[i], out[i], {"x": obs[i][0], "y": obs[i][1]})
            cnt += 1
mean_loss = np.mean(test_loss)
print("Epoch Mean Test Loss: {}".format(mean_loss))

In [ ]:
traj_inp.numpy()[0][:2*num].shape, traj_out.numpy()[0].shape

In [ ]:
traj_inp[6]

In [ ]:
traj_out[6]

In [ ]:
out[6]

In [ ]:
torch.sqrt(torch.sum((traj_out[6] - out[6]) ** 2))

In [ ]:
fin = torch.tensor([traj_out[6][1], traj_out[6][23]])
fin_pred = torch.tensor([out[6][11], out[6][23]])
fin, fin_pred

In [ ]:
torch.sqrt(torch.sum((fin_pred - fin) ** 2))

In [ ]:
torch.sqrt(torch.sum((traj_out[6][-1] - out[6][-1]) ** 2))

In [ ]:
criterion(traj_out[6], out[6])

In [ ]:
def disp_traj(cnt, traj_inp, traj_out, traj_pred):
    traj_inp = traj_inp.numpy()
    traj_out = traj_out.numpy()
    traj_pred = traj_pred.numpy()
    
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.scatter(traj_inp[::2], traj_inp[1::2], label='Inp traj')
    ax.scatter(traj_out[:12], traj_out[12:], label='GT')
    ax.scatter(traj_out[11], traj_out[23], label='Fin GT')
    ax.scatter(traj_pred[:12], traj_pred[12:], label='Pred')
    ax.scatter(traj_pred[11], traj_pred[23], label='Fin Pred')
    
    th = np.linspace(0, 2 * np.pi, 100)
    for i in range(0, num_obs):
        x_circ = x_obs_temp[i] + a_obs * np.cos(th)
        y_circ = y_obs_temp[i] + b_obs * np.sin(th)
        ax.plot(x_circ, y_circ, '-k')
    
    ax.legend()
    ax.set_xlim([-7, 7])
    ax.set_ylim([-7, 7])
    plt.show()

In [ ]:
disp_traj(0, traj_inp[8], traj_out[8], out[8].detach())

In [ ]:
import cvxopt
class OPTNodee(AbstractDeclarativeNode):
    def __init__(self, P, Pddot, A_eq, A_obs, Q_smoothness, x_obs, y_obs, num=12, num_obs=4, nvar=11, a_obs=1.0, b_obs=1.0, rho_obs=0.3, rho_eq=10.0, weight_smoothness=10, maxiter=300, eps=1e-7, num_tot=48):
        super().__init__()
        self.P = torch.tensor(P, dtype=torch.double).to(device)
        self.Pddot = torch.tensor(Pddot, dtype=torch.double).to(device)
        self.A_eq = torch.tensor(A_eq, dtype=torch.double).to(device)
        self.A_obs = torch.tensor(A_obs, dtype=torch.double).to(device)
        self.Q_smoothness = torch.tensor(Q_smoothness, dtype=torch.double).to(device)
        self.x_obs = torch.tensor(x_obs, dtype=torch.double).to(device)
        self.y_obs = torch.tensor(y_obs, dtype=torch.double).to(device)
        
        self.num = num
        self.num_obs = num_obs
        self.eps = eps
        self.nvar = nvar        
        self.a_obs = a_obs
        self.b_obs = b_obs        
        self.rho_eq = rho_eq
        self.num_obs = num_obs
        self.maxiter = maxiter
        self.num_tot = num_tot
        self.rho_obs = rho_obs
        self.weight_smoothness = weight_smoothness
        
    def objective(self, b, lamda_x, lamda_y, y):  
        batch_size, _ = b.size()
        b = b.transpose(0, 1)
        y = y.transpose(0, 1)
        lamda_x = lamda_x.transpose(0, 1)
        lamda_y = lamda_y.transpose(0, 1)
        bx_eq_tensor, by_eq_tensor = torch.split(b, 6, dim=0)
        ones_tensor = torch.ones(self.num_tot, batch_size, dtype=torch.double).to(device)

        c_x = y[0:self.nvar]
        c_y = y[self.nvar:2 * self.nvar]
        alpha_obs = y[2 * self.nvar: 2 * self.nvar + self.num_tot]
        d_obs = y[2 * self.nvar + self.num_tot:]

        cost_smoothness_x = 0.5 * self.weight_smoothness * torch.diag(torch.matmul(c_x.T, torch.matmul(self.Q_smoothness, c_x)))
        cost_smoothness_y = 0.5 * self.weight_smoothness * torch.diag(torch.matmul(c_y.T, torch.matmul(self.Q_smoothness, c_y)))

        temp_x_obs = d_obs * torch.cos(alpha_obs) * self.a_obs
        b_obs_x = self.x_obs.view(-1, 1) + temp_x_obs

        temp_y_obs = d_obs * torch.sin(alpha_obs) * self.b_obs
        b_obs_y = self.y_obs.view(-1, 1) + temp_y_obs

        cost_obs_x = 0.5 * self.rho_obs * (torch.sum((torch.matmul(self.A_obs, c_x) - b_obs_x) ** 2, axis=0))
        cost_obs_y = 0.5 * self.rho_obs * (torch.sum((torch.matmul(self.A_obs, c_y) - b_obs_y) ** 2, axis=0))
        cost_slack = self.rho_obs * torch.sum(torch.max(1 - d_obs, ones_tensor), axis=0)

        cost_eq_x = 0.5 * self.rho_eq * torch.sum((torch.matmul(self.A_eq, c_x) - bx_eq_tensor) ** 2, axis=0)
        cost_eq_y = 0.5 * self.rho_eq * torch.sum((torch.matmul(self. A_eq, c_y) - by_eq_tensor) ** 2, axis=0)

        cost_x = cost_smoothness_x + cost_obs_x + cost_eq_x
        cost_y = cost_smoothness_y + cost_obs_y + cost_eq_y
        cost = cost_x + cost_y + self.eps * torch.sum(c_x ** 2, axis=0) + self.eps * torch.sum(c_y ** 2, axis=0) + self.eps * torch.sum(d_obs ** 2, axis=0) + self.eps * torch.sum(alpha_obs ** 2, axis=0) + cost_slack
        return cost

    def optimize(self, b, lamda_x, lamda_y):
        bx_eq_tensor, by_eq_tensor = torch.split(b, 6, dim=0)
        
        d_obs = torch.ones(self.num_obs, self.num, dtype=torch.double).to(device)
        alpha_obs = torch.zeros(self.num_obs, self.num, dtype=torch.double).to(device)
        ones_tensor = torch.ones((self.num_obs, self.num), dtype=torch.double).to(device)
        cost_smoothness = self.weight_smoothness * torch.matmul(self.Pddot.T, self.Pddot)
        cost = cost_smoothness + self.rho_obs * torch.matmul(self.A_obs.T, self.A_obs) + self.rho_eq * torch.matmul(self.A_eq.T, self.A_eq)

        temp_x_obs = d_obs * torch.cos(alpha_obs) * self.a_obs
        temp_y_obs = d_obs * torch.sin(alpha_obs) * self.b_obs

        b_obs_x = self.x_obs.view(self.num * self.num_obs) + temp_x_obs.view(self.num * self.num_obs)
        b_obs_y = self.y_obs.view(self.num * self.num_obs) + temp_y_obs.view(self.num * self.num_obs)        
        
        bx_eq_tensor, by_eq_tensor = torch.split(b, 6, dim=0)
        
#         + torch.matmul(self.A_obs.T, b_obs_x) + self.rho_eq * torch.matmul(self.A_eq.T, bx_eq_tensor)
#          self.rho_obs * torch.matmul(self.A_obs.T, b_obs_y) - self.rho_eq * torch.matmul(self.A_eq.T, by_eq_tensor)        
        
#       A_eq = np.vstack((P[0], Pdot[0], Pddot[0], P[-1], Pdot[-1], Pddot[-1]))
#       A_eq = np.vstack((A_eq, A_eq))
#        A_obs = np.tile(P, (num_obs, 1))
        b_eq = np.hstack((bx_eq_tensor, by_eq_tensor))
        print(A_obs.shape)
#         A_eq = np.vstack((P[0], Pdot[0], Pddot[0], P[-1], Pdot[-1], Pddot[-1]))
        Q_obs = torch.matmul(self.A_obs.T, self.A_obs)
        Q_eq = torch.matmul(self.A_eq.T, self.A_eq)
        bx_obs = x_obs.reshape(num_obs * num)
        by_obs = y_obs.reshape(num_obs * num)
        qx_obs = torch.matmul(b_obs_x.T, self.A_obs) * 2
        qy_obs = torch.matmul(b_obs_y.T, self.A_obs) * 2
        qx_eq = torch.matmul(bx_eq_tensor.T, self.A_eq) * 2
        qy_eq = torch.matmul(by_eq_tensor.T, self.A_eq) * 2
        q = np.hstack((qx_eq, qy_eq))
        print(Q_obs.shape)
        AA_eq = cvxopt.matrix(self.A_eq, tc='d')
        bx_eq = cvxopt.matrix(bx_eq_tensor, tc='d')
        by_eq = cvxopt.matrix(by_eq_tensor, tc='d')
        b_eq = cvxopt.matrix(b_eq, tc='d')
        Q = cvxopt.matrix(cost, tc='d')
        q_x = cvxopt.matrix(rho_obs * qx_obs + rho_eq * qx_eq)
        q_y = cvxopt.matrix(rho_obs * qy_obs + rho_eq * qy_eq)
        sol_x = solvers.qp(Q, q_x, None, None, AA_eq, bx_eq)
        sol_x = torch.tensor(sol_x['x'])
        sol_y = solvers.qp(Q, q_y, None, None, AA_eq, by_eq)
        sol_y = torch.tensor(sol_y['x'])
        sol = torch.cat([sol_x, sol_y, alpha_obs.view(-1), d_obs.view(-1)])
        return sol

    def solve(self, b, lamda_x, lamda_y):
        batch_size, _ = b.size()
        b = b.transpose(0, 1)
        lamda_x = lamda_x.transpose(0, 1)
        lamda_y = lamda_y.transpose(0, 1)
        y = torch.zeros(batch_size, 2 * self.nvar + 2 * self.num_tot, dtype=torch.double).to(device)
        for i in range(batch_size):
            b_cur = b[:, i]
            lamda_x_cur = lamda_x[:, i]
            lamda_y_cur = lamda_y[:, i]
            sol = self.optimize(b_cur, lamda_x_cur, lamda_y_cur)
            y[i, :] = sol
        return y, None